In [1]:
import piplite

In [2]:
await piplite.install(['folium'])

In [3]:
await piplite.install(['pandas'])

In [4]:
import folium
import pandas as pd

In [5]:
# Import folium MarkerCluster plugin
from folium.plugins import MarkerCluster
# Import folium MousePosition plugin
from folium.plugins import MousePosition
# Import folium DivIcon plugin
from folium.features import DivIcon

In [13]:
# Download and read the `spacex_launch_geo.csv`
from js import fetch
import io

URL = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv'
resp = await fetch(URL)
spacex_csv_file = io.BytesIO((await resp.arrayBuffer()).to_py())
spacex_df=pd.read_csv(spacex_csv_file)

In [15]:
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]

In [16]:
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]

In [17]:
# Crear mapa
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)

# Añadir círculos y etiquetas para cada launch site
for index, row in launch_sites_df.iterrows():
    lat = row['Lat']
    lon = row['Long']
    site_name = row['Launch Site']

    # Círculo
    folium.Circle(
        [lat, lon],
        radius=1000,
        color='#d35400',
        fill=True
    ).add_child(folium.Popup(site_name)).add_to(site_map)

    # Etiqueta
    folium.Marker(
        [lat, lon],
        icon=DivIcon(
            icon_size=(20, 20),
            icon_anchor=(0, 0),
            html=f'<div style="font-size: 12; color:#d35400;"><b>{site_name}</b></div>'
        )
    ).add_to(site_map)

site_map


In [18]:
# Definir color según éxito o fallo
spacex_df['marker_color'] = spacex_df['class'].apply(lambda x: 'green' if x == 1 else 'red')

marker_cluster = MarkerCluster()

for _, record in spacex_df.iterrows():
    marker = folium.Marker(
        [record['Lat'], record['Long']],
        icon=folium.Icon(color=record['marker_color'])
    )
    marker_cluster.add_child(marker)

site_map.add_child(marker_cluster)
site_map


In [20]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # radio aproximado de la Tierra en km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance



In [21]:
# Coordenadas del launch site y de la costa
launch_lat = 28.563197
launch_lon = -80.576820
coast_lat = 28.56367
coast_lon = -80.57163

# Calcular distancia
distance_coast = calculate_distance(launch_lat, launch_lon, coast_lat, coast_lon)

# Marcar la costa
distance_marker = folium.Marker(
    [coast_lat, coast_lon],
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>{:.2f} KM</b></div>'.format(distance_coast)
    )
)
site_map.add_child(distance_marker)

# Dibujar línea entre el sitio y la costa
lines = folium.PolyLine(
    locations=[[launch_lat, launch_lon], [coast_lat, coast_lon]],
    weight=2
)
site_map.add_child(lines)

site_map
